In [3]:
import math
import random
import pygame
import sys

# Size of canvas. These get updated to fill the whole screen.
width = 1000
height = 750
boidneigbor = 3
numBoids = 100
visualRange = 75

boids = []
 
def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1

    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005

    centerX = 0
    centerY = 0
    numNeighbors = 0

    for otherBoid in boids:
        # if otherBoid in nClosestBoids(boid,boidneighbor):
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors

        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0

    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05

    avgDX = 0
    avgDY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
        #if otherBoid in nClosestBoids(boid,boidneighbor):
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors

        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 15

    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, (85, 140, 244), [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)

    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, (85, 140, 244, 102), (int(point[0]), int(point[1])), 1)

# Main animation loop
def animationLoop():
    global boids
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

    screen.fill((255, 255, 255))
    for boid in boids:
        drawBoid(screen, boid)

    pygame.display.flip()
    pygame.time.Clock().tick(60)

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = False



pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
initBoids()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    animationLoop()

In [4]:
import json
num_time_steps = 100
num_sims = 10
velocities = []
positions =[]


for k in range(num_sims): # run simulation num_sims times
    initBoids() # initialize boids with different positions, and velocities
    for _ in range(num_time_steps): # run animationLoop() for num_time_steps
        for event in pygame.event.get(): # code to quit the animation (if needed)
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        animationLoop() # updates boids to have new positions and velocities

    time_list = []  # Ensure time_list is initialized

    for i in range(len(boids[1]["history"])):  # Go through the length of history
        boid_dict = {}  # Initialize the dictionary for the current time step
        if i == 0:
            continue
        else: 
            for j in range(len(boids)):  # Go through the number of boids for every second
                dic = {}  # Initialize the dictionary for the current boid
                dic['x'] = boids[j]['history'][i][0]
                dic['y'] = boids[j]['history'][i][1]
                dic['dx'] = boids[j]['history'][i][0] - boids[j]['history'][i-1][0]
                dic['dy'] = boids[j]['history'][i][1] - boids[j]['history'][i-1][1]

                boid_dict[f'boid{j}'] = dic

        time_list.append(boid_dict)  # Append the dictionary to the list
    
    jake = json.dumps(time_list)

    with open(f"../data/myjson/mydata{k}.json", "w") as final:
        json.dump(jake, final)



In [5]:
time_list

[{'boid0': {'x': 583.4093658025771,
   'y': 314.34316842174496,
   'dx': 8.560835520232217,
   'dy': 1.020857300451496},
  'boid1': {'x': 435.7716261293276,
   'y': 299.2292545537857,
   'dx': 7.156180688900292,
   'dy': 0.8111246386442303},
  'boid2': {'x': 456.18523353755086,
   'y': 338.5193930174208,
   'dx': 7.121593610219463,
   'dy': 0.48029708906710766},
  'boid3': {'x': 577.0372520559307,
   'y': 558.6041115335845,
   'dx': -3.426332234229676,
   'dy': -0.8217692978385003},
  'boid4': {'x': 592.8842348193635,
   'y': 387.63603511664127,
   'dx': 8.02970539347109,
   'dy': 3.303337921378443},
  'boid5': {'x': 555.911286767111,
   'y': 473.48860232673775,
   'dx': -7.250512299727461,
   'dy': 4.766857445221262},
  'boid6': {'x': 505.129842111337,
   'y': 353.85363764712935,
   'dx': -2.7694359638688297,
   'dy': -0.17596293159516563},
  'boid7': {'x': 588.6738333160944,
   'y': 353.6734570817346,
   'dx': 4.840258215746758,
   'dy': 2.962777324289675},
  'boid8': {'x': 482.36863